In [51]:
import torch
import torch_mlir

In [52]:
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)

In [54]:
model.eval()

compiled_module = torch_mlir.compile(model, example_args=[torch.ones((1, 3, 224, 224))], output_type=torch_mlir.OutputType.TOSA)

from torch_mlir_e2e_test.tosa_backends.linalg_on_tensors import LinalgOnTensorsTosaBackend
from torch_mlir_e2e_test.linalg_on_tensors_backends.refbackend import RefBackendLinalgOnTensorsBackend
# backend = LinalgOnTensorsTosaBackend()
backend = RefBackendLinalgOnTensorsBackend()
compiled_module = torch_mlir.compile(model, example_args=[torch.ones((1, 3, 224, 224))], output_type=torch_mlir.OutputType.LINALG_ON_TENSORS)
runnable = backend.compile(compiled_module)
jit_module = backend.load(runnable)


In [55]:
outfile = open("resnet18.mlir", "w")
outfile.write(str(compiled_module))

94063946

In [56]:
# get example image
from PIL import Image
import requests
from io import BytesIO
from torchvision import transforms

def load_and_preprocess_image(url: str):
    headers = {
        'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    }
    img = Image.open(requests.get(url, headers=headers,
                                  stream=True).raw).convert("RGB")
    # preprocessing pipeline
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    img_preprocessed = preprocess(img)
    return torch.unsqueeze(img_preprocessed, 0)

image_url = "https://upload.wikimedia.org/wikipedia/commons/2/26/YellowLabradorLooking_new.jpg"


img = load_and_preprocess_image(image_url)

print(img.shape)
# write img to file
torch.save(img.numpy(), "../examples/test_image.pt")
# write to json 
import json
with open("../examples/test_image.json", "w") as f:
    json.dump({
        "data": img.numpy().flatten().tolist(), 
        "shape": list(img.shape)},f)
arg = img.numpy()

torch.Size([1, 3, 224, 224])


In [57]:
torch.__version__

'2.0.0.dev20230209+cu117'

In [58]:
import timeit

timeit.timeit(lambda: jit_module.forward(arg), number=1)

16.245480597004644

In [59]:

timeit.timeit(lambda: jit_module.forward(arg), number=1)

2.6393603989999974

In [ ]:

timeit.timeit(lambda: model(img), number=1)

0.06698099699860904